In [39]:
import pandas as pd
from indicators import *

# Load the CSV data
df = pd.read_csv('./data/btc_1h_data_training.csv')

# Initialize dataframes for trends
uptrend_df = pd.DataFrame()
downtrend_df = pd.DataFrame()
sideways_df = pd.DataFrame()

df.head(5)

,time,open,high,low,close,volume
0,2018-01-01 00:00:00,13715.65,13715.65,13400.01,13529.01,443.356199
1,2018-01-01 01:00:00,13528.99,13595.89,13155.38,13203.06,383.697006
2,2018-01-01 02:00:00,13203.00,13418.43,13200.00,13330.18,429.064572
3,2018-01-01 03:00:00,13330.26,13611.27,13290.00,13410.03,420.087030
4,2018-01-01 04:00:00,13434.98,13623.29,13322.15,13601.01,340.807329


In [40]:
from utils import Normalizing


df = df.rename(columns={'time': 'Timestamp', 'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close',
                            'volume': 'Volume'})
df = AddIndicators(df)
df = df[100:]
df_original = df.copy()
df_original = df_original.iloc[8:]

df = Normalizing(df).dropna()
df['MA'] = df['Close'].rolling(window=7).mean()
df = df.iloc[7:]
print(len(df))
print(len(df_original))
print((df.head(5)))
print((df_original.head(5)))

54515
54515
               Timestamp     Close       rsi       cmf      natr        MA
108  2018-01-05 13:00:00  0.600763  0.687982  0.239490  0.022683  0.585486
109  2018-01-05 14:00:00  0.557984  0.689219  0.219465  0.022674  0.586565
110  2018-01-05 15:00:00  0.553737  0.684330  0.211637  0.021916  0.573033
111  2018-01-05 16:00:00  0.617861  0.732149  0.282888  0.021694  0.570305
112  2018-01-05 17:00:00  0.590010  0.754437  0.276969  0.022896  0.568739
               Timestamp      Open      High       Low     Close       Volume  \
108  2018-01-05 13:00:00  15838.00  16199.00  15750.00  16093.94   996.320352   
109  2018-01-05 14:00:00  16071.62  16267.04  15901.01  16103.18   991.739170   
110  2018-01-05 15:00:00  16103.18  16170.00  15980.63  16087.72   787.319320   
111  2018-01-05 16:00:00  16094.99  16500.00  16087.73  16449.00  1007.615517   
112  2018-01-05 17:00:00  16449.00  17118.13  16420.00  16650.00   937.578122   

           rsi         atr       cmf  
108  68.7981

In [42]:
# Loop through the dataset in chunks of 6 rows
for i in range(5, len(df)):  # Start from the 6th row because of the moving average window
    # Get the moving average values for the chunk
    ma_chunk = df.iloc[i-5:i+1]['MA']

    # Check if the chunk has valid moving averages
    if ma_chunk.isna().any():
        continue

    # Determine the trend based on the moving average
    if ma_chunk.is_monotonic_increasing:  # If moving average is increasing
        uptrend_df = pd.concat([uptrend_df, df.iloc[i-5:i+1]])
    elif ma_chunk.is_monotonic_decreasing:  # If moving average is decreasing
        downtrend_df = pd.concat([downtrend_df, df.iloc[i-5:i+1]])
    else:  # If neither increasing nor decreasing, it's sideways
        sideways_df = pd.concat([sideways_df, df.iloc[i-5:i+1]])

In [43]:
uptrend_df.to_csv('./data/uptrend_normalised.csv', index=False)
downtrend_df.to_csv('./data/downtrend_normalised.csv', index=False)
sideways_df.to_csv('./data/sideways_normalised.csv', index=False)

In [44]:
print("uptrend_df length: ", len(uptrend_df))
print("downtrend_df length: ", len(downtrend_df))
print("sideways_df length: ", len(sideways_df))
total = len(uptrend_df) + len(downtrend_df) + len(sideways_df)
print("uptrend_df / total length: ", len(uptrend_df)/total)
print("downtrend_df / total length: ", len(downtrend_df)/total)
print("sideways_df / total length: ", len(sideways_df)/total)

uptrend_df length:  7140
downtrend_df length:  6978
sideways_df length:  312942
uptrend_df / total length:  0.021830856723536966
downtrend_df / total length:  0.021335534764263437
sideways_df / total length:  0.9568336085121996


In [45]:
import pandas as pd
from indicators import *

# Load the CSV data
uptrend_df = pd.read_csv('./data/uptrend_normalised.csv')
downtrend_df = pd.read_csv('./data/downtrend_normalised.csv')
sideways_df = pd.read_csv('./data/sideways_normalised.csv')

In [47]:
sideways_df = sideways_df.iloc[305_964:]
print(sideways_df.head(7))

                  Timestamp     Close       rsi       cmf      natr        MA
305964  2024-02-09 20:00:00  0.556501  0.714757  0.017324  0.008666  0.556773
305965  2024-02-09 21:00:00  0.556160  0.713485  0.017720  0.008519  0.560118
305966  2024-02-09 22:00:00  0.542692  0.642010 -0.040673  0.008730  0.557339
305967  2024-02-09 23:00:00  0.548196  0.603225 -0.011817  0.008472  0.555197
305968  2024-02-10 00:00:00  0.553765  0.590917 -0.013632  0.008186  0.551686
305969  2024-02-10 01:00:00  0.568946  0.629801 -0.025832  0.008011  0.553896
305970  2024-02-09 21:00:00  0.556160  0.713485  0.017720  0.008519  0.560118


In [48]:
# min_length = min(len(uptrend_df), len(downtrend_df), len(sideways_df))

# Sample the same number of rows from each DataFrame
# uptrend_sampled = uptrend_df.sample(n=min_length, random_state=42)
# downtrend_sampled = downtrend_df.sample(n=min_length, random_state=42)
# sideways_sampled = sideways_df.sample(n=min_length, random_state=42)


# Concatenate the sampled DataFrames
balanced_df = pd.concat([uptrend_df, downtrend_df, sideways_df])

# Reset the index if necessary
balanced_df = balanced_df.reset_index(drop=True)

# Save the balanced DataFrame to a CSV file
balanced_df.to_csv('./data/balanced_trend_data_normalised.csv', index=False)

print("Balanced DataFrame created with equal distribution of trends.")

Balanced DataFrame created with equal distribution of trends.


In [54]:
balanced_df = balanced_df.drop(balanced_df.columns[1:], axis=1)
result_df = balanced_df.merge(df_original, on='Timestamp', how='inner')
result_df.head(10)
result_df.to_csv('./data/balanced_trend_data_normalised_original.csv', index=False)